In [ ]:
#@title Importamos el Dataset { run: "auto" }
!kaggle datasets download -d manideep1108/tusimple

In [ ]:
!unzip tusimple.zip

In [ ]:
import os
import matplotlib.pyplot as plt
from IPython.display import Image
import keras
from keras.preprocessing import image
from keras.utils import plot_model
import numpy as np
import pandas as pd
import scipy
import glob
from tqdm.notebook import tqdm
import json
import cv2

In [ ]:
#@title Dataset{ run: "auto" }
filenames = np.array(glob.glob('TUSimple/train_set/**/*.jpg', recursive=True))

In [ ]:
import os
import json
import glob

# Ruta a las imágenes
image_path = './TUSimple/train_set'

# Obtener la lista de archivos JSON de anotaciones
annotation_paths = glob.glob(os.path.join(image_path, 'label_data_*.json'))
print("Archivos JSON de anotaciones encontrados:", annotation_paths)

# Lista para almacenar los nombres de las imágenes del JSON
json_image_names = []

# Cargar nombres de imágenes desde todos los archivos JSON
for annotation_path in annotation_paths:
    with open(annotation_path, 'r') as f:
        try:
            # Leer todas las líneas del archivo
            lines = f.readlines()

            # Iterar sobre cada línea y cargar el JSON
            for line in lines:
                try:
                    data = json.loads(line)
                    # Suponiendo que hay una clave específica en cada JSON que contiene los nombres de las imágenes
                    image_name = os.path.basename(data['raw_file'])
                    json_image_names.append(image_name)
                except json.JSONDecodeError as e:
                    print(f"Error al decodificar JSON en {annotation_path}: {e}")

        except Exception as ex:
            print(f"Error al procesar {annotation_path}: {ex}")

print("Total de nombres de imágenes en JSON:", len(json_image_names))

# Patrón para buscar imágenes solo en las carpetas deseadas dentro de 'image_path'
image_folders = ['clips/0531', 'clips/0601']  # Agrega los nombres de las carpetas deseadas aquí

image_files = []
for folder in image_folders:
    folder_pattern = os.path.join(image_path, folder, '**', '*.jpg')
    image_files.extend(glob.glob(folder_pattern, recursive=True))

# Filtrar las imágenes reales para incluir solo las que están en json_image_names
img_files = [img_path for img_path in image_files if os.path.basename(img_path) in json_image_names]

print("\nTotal de imágenes encontradas en los JSON:", len(img_files))
print("Ejemplo de imágenes encontradas:")
for img_path in img_files[:5]:  # Mostrar los primeros 5 ejemplos
    print(img_path)


In [ ]:
annotations = []
for annotation_path in annotation_paths:
    with open(annotation_path, 'r') as f:
        for line in f:
            annotations.append(json.loads(line))

In [ ]:
#@title Preparamos el dataset { run: "auto" }
import numpy as np
from tensorflow.keras.utils import Sequence
import cv2

class LaneDataset(Sequence):
    def __init__(self, image_files, annotations, batch_size=32, image_size=(512, 256)):
        self.image_files = image_files
        self.annotations = annotations
        self.batch_size = batch_size
        self.image_size = image_size

    def __len__(self):
        return int(np.ceil(len(self.image_files) / self.batch_size))

    def preprocess_image(self, image_path):
        image = cv2.imread(image_path)
        image = cv2.resize(image, self.image_size)
        image = image / 255.0  # Normalizar la imagen
        return image

    def extract_lane_points(self, annotation):
        # Implementa esta función según cómo estén estructuradas tus anotaciones
        return annotation['lanes']

    def pad_lanes(self, lanes, max_points=100):
        padded_lanes = np.zeros((len(lanes), max_points, 2))  # Suponiendo puntos de carril (x, y)
        for i, lane in enumerate(lanes):
            num_points = min(len(lane), max_points)
            padded_lanes[i, :num_points, 0] = lane[:num_points, 0]
            padded_lanes[i, :num_points, 1] = lane[:num_points, 1]
        return padded_lanes


    def __getitem__(self, idx):
      batch_image_files = self.image_files[idx * self.batch_size:(idx + 1) * self.batch_size]
      batch_annotations = self.annotations[idx * self.batch_size:(idx + 1) * self.batch_size]

      images = np.array([self.preprocess_image(f) for f in batch_image_files])
      lanes = []
      for ann in batch_annotations:
          lane_points = self.extract_lane_points(ann)
          lane_array = np.array(lane_points)  # Convert to array numpy if not lo es
          lanes.append(lane_array)

      if not isinstance(images, np.ndarray):
          raise TypeError(f"Images is not a numpy array: {images}")
      if not isinstance(lanes, list):
          raise TypeError(f"Lanes is not a list: {lanes}")

      images = np.array(images)
      lanes = self.pad_lanes(lanes)  # Call the function pad_lanes with lanes as list of arrays numpy

      return images, lanes

# Crear el dataset
dataset = LaneDataset(image_files, annotations)




In [ ]:
from sklearn.model_selection import train_test_split

train_images, val_images, train_annotations, val_annotations = train_test_split(img_files, annotations, test_size=0.2, random_state=42)

train_dataset = LaneDataset(train_images, train_annotations)
val_dataset = LaneDataset(val_images, val_annotations)

In [ ]:
#@title Modelo de entrenamiento base{ run: "auto" }
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, BatchNormalization, Activation

def LaneNet(input_shape=(256, 512, 3)):
    inputs = Input(shape=input_shape)

    # Encoder
    conv1 = Conv2D(64, 3, padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    conv1 = Conv2D(64, 3, padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, padding='same')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    conv2 = Conv2D(128, 3, padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, padding='same')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)
    conv3 = Conv2D(256, 3, padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, padding='same')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Activation('relu')(conv4)
    conv4 = Conv2D(512, 3, padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    conv4 = Activation('relu')(conv4)

    # Decoder
    up5 = UpSampling2D(size=(2, 2))(conv4)
    up5 = Concatenate()([up5, conv3])
    conv5 = Conv2D(256, 3, padding='same')(up5)
    conv5 = BatchNormalization()(conv5)
    conv5 = Activation('relu')(conv5)
    conv5 = Conv2D(256, 3, padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)
    conv5 = Activation('relu')(conv5)

    up6 = UpSampling2D(size=(2, 2))(conv5)
    up6 = Concatenate()([up6, conv2])
    conv6 = Conv2D(128, 3, padding='same')(up6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Activation('relu')(conv6)
    conv6 = Conv2D(128, 3, padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Activation('relu')(conv6)

    up7 = UpSampling2D(size=(2, 2))(conv6)
    up7 = Concatenate()([up7, conv1])
    conv7 = Conv2D(64, 3, padding='same')(up7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Activation('relu')(conv7)
    conv7 = Conv2D(64, 3, padding='same')(conv7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Activation('relu')(conv7)

    # Output layer
    output = Conv2D(1, 1, activation='sigmoid')(conv7)

    model = Model(inputs=inputs, outputs=output)

    return model

# Crear y compilar el modelo
model = LaneNet()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()



In [ ]:
model = LaneNet()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_dataset, validation_data=val_dataset, verbose=3, epochs=20)